## AgriChemCo Distribution Optimization Case Study

In [1]:
import pandas as pd
import numpy as np

cap = pd.read_csv("cap.csv", index_col=0)
demand = pd.read_csv("demand.csv", index_col=0)
route_cost_prod = pd.read_csv("cost.csv", index_col=0)

route_cost_prod.index = [f"{i[0]}{i[-1]}" for i in route_cost_prod.index]

dfs = [cap, route_cost_prod, demand]

names = ["CAPACITY (UNITS)", "ROUTE COSTS ($)", "DEMAND (UNITS)"]

for i, df in enumerate(dfs):
    df.index.name = names[i]

### Context
You are the logistics leader at AgriChemCo, an agrochemical company specializing in the production and distribution of agricultural chemicals. The company aims to optimize the distribution of its products - Fertilizer (F), Pesticide (P), and Herbicide (H) - from manufacturing plants to various regional warehouses, ensuring that the demand at each warehouse is met at the lowest possible transportation cost.

The relevant parameters are defined as:

\begin{align*}
&   o   \in \{A, B, C\},    \\
&   d   \in \{N, S, E, W\}, \\
&   p   \in \{F,P,H\}       \\
\end{align*}

where:
- $o$ represents one of three production plants
- $d$ represents one of four warehouse locations
- $p$ represents one of three types of products

`cap` tells us the capacity of each plant $o$ in the production of each product $p$:

In [2]:
cap

,F,P,H
CAPACITY (UNITS),,,
A,600,500,400
B,300,600,300
C,500,400,500


`route_cost_prod` contains data that tell us how much it costs to ship a product from plant $o$ to warehouse $d$:

In [3]:
route_cost_prod

,F,P,H
ROUTE COSTS ($),,,
AN,2.0,2.5,3.0
AS,3.0,2.0,2.5
AE,2.5,3.0,2.0
AW,4.0,3.5,4.0
BN,3.0,3.5,3.0
BS,2.0,2.0,4.0
BE,4.0,4.0,2.5
BW,3.5,2.5,3.5
CN,2.5,2.0,2.5


`demand` tells us the demand at each warehouse for each product:

In [4]:
demand

,F,P,H
DEMAND (UNITS),,,
N,400,300,500
S,500,600,200
E,300,200,400
W,200,400,100


### Objective
We want to minimize the total transportation cost of distributing Fertilizer (F), Pesticide (P), and Herbicide (H) from manufacturing plants to regional warehouses.

### Solution

$$
\begin{align*}

\min        \quad   &   \sum^{3}_{o=1} \sum^{4}_{d=1} \sum^{3}_{p=1}  C_{odp}X_{odp} \\
\text{s.t.} \quad   &   

\end{align*}
$$

where:
- o = index of the plant
- d = index of the demand region
- p = index of the product type

In [5]:
from pulp import *

plants = cap.index
locs = demand.index
prods = cap.columns

problem = LpProblem("minimize_transportation_costs", LpMinimize)

x = LpVariable.dicts(
    "quantity", ((o, d, p) for o in plants for d in locs for p in prods), 
    lowBound=0,
    cat="Integer"
)

problem += lpSum([
    route_cost_prod.loc[f"{o}{d}", p] * x[(o, d, p)] \
        for o in plants \
            for d in locs \
                for p in prods
])

for o in plants:
    for p in prods:
        problem += lpSum([x[o, d, p] for d in locs]) <= cap.loc[o, p]

for d in locs:
    for p in prods:
        problem += lpSum([x[o, d, p] for o in plants]) >= demand.loc[d, p]

problem.solve()

if problem.status == 1:
    print(LpStatus[problem.status])
    quantities = pd.DataFrame()
    for o, d, p in x.keys():
        quantities.loc[f"{o}{d}", p] = x[o, d, p].varValue
    quantities.index.name = "OPTIMAL QUANTITY (UNITS)"
else:
    print(LpStatus[problem.status])

Optimal


In [6]:
print(quantities)

                              F      P      H
OPTIMAL QUANTITY (UNITS)                     
AN                        400.0    0.0    0.0
AS                        200.0  300.0  200.0
AE                          0.0  200.0  200.0
AW                          0.0    0.0    0.0
BN                          0.0    0.0  100.0
BS                        300.0  300.0    0.0
BE                          0.0    0.0  200.0
BW                          0.0  300.0    0.0
CN                          0.0  300.0  400.0
CS                          0.0    0.0    0.0
CE                        300.0    0.0    0.0
CW                        200.0  100.0  100.0


In [7]:
print(f"Total cost: ${problem.objective.value():,.2f}")

Total cost: $9,600.00


In [8]:
total_prod_quantities = {p: [] for p in prods}

for p in prods:
    for o in plants:
        for d in locs:
            total_prod_quantities[p].append(x[o, d, p].varValue)
    total_prod_quantities[p] = np.sum(total_prod_quantities[p])

total_prod_quantities

{'F': 1400.0, 'P': 1500.0, 'H': 1200.0}

In [9]:
total_prod_costs = np.sum(quantities.values * route_cost_prod.values, axis=0)
total_prod_costs = {prods[i]: f"${total_prod_costs[i]:,.2f}" for i in range(len(prods))}

total_prod_costs

{'F': '$3,300.00', 'P': '$3,400.00', 'H': '$2,900.00'}

In [10]:
demand_quantity = {d: {p: [] for p in prods} for d in locs}

for d in locs:
    for o in plants:
        for p in prods:
            demand_quantity[d][p].append(quantities.loc[f"{o}{d}", p])
            demand_quantity[d][p] = [np.sum(demand_quantity[d][p])]

pd.DataFrame(demand_quantity).T.applymap(lambda x: x[0])

,F,P,H
N,400.0,300.0,500.0
S,500.0,600.0,200.0
E,300.0,200.0,400.0
W,200.0,400.0,100.0
